In [ ]:
# Importando as libs
import os
import requests
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

In [11]:

# Configuração do PostgreSQL
DB_CONFIG = {
    "dbname": "seguranca_publica",
    "user": "seu_usuario",
    "password": "sua_senha",
    "host": "localhost",
    "port": "5432",
}

In [ ]:
# Configuração do Datajud
API_KEY = os.environ.get("DATAJUD_API_KEY")
if not API_KEY:
    raise ValueError("Chave de API do Datajud não encontrada nas variáveis de ambiente.")

ValueError: Chave de API do Datajud não encontrada nas variáveis de ambiente.

In [8]:
# Configuração da API

API_URL = "https://api-publica.datajud.cnj.jus.br/api_publica_trf1/"
API_KEY = "cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw=="  # Sua chave de API

headers = {
    "Authorization": f"APIKey {API_KEY}"
}

def extrair_dados_api():
    response = requests.get(API_URL, headers=headers)

    if response.status_code == 200:
        dados = response.json()
        return dados
    else:
        print(f"Erro ao acessar API: {response.status_code}")
        print(response.text) # Imprime o conteúdo da resposta para auxiliar na depuração
        return None

dados_api = extrair_dados_api()

if dados_api:
    print(dados_api)

Erro ao acessar API: 403
{"error":{"root_cause":[{"type":"security_exception","reason":"action [indices:admin/get] is unauthorized for API key id [p6Gc9YkBZuTDd2BzwPmv] of user [dpj_api_publica] on indices [api_publica_trf1], this action is granted by the index privileges [view_index_metadata,manage,all]"}],"type":"security_exception","reason":"action [indices:admin/get] is unauthorized for API key id [p6Gc9YkBZuTDd2BzwPmv] of user [dpj_api_publica] on indices [api_publica_trf1], this action is granted by the index privileges [view_index_metadata,manage,all]"},"status":403}


In [ ]:
# Função para transformar os dados em um DataFrame do Pandas
def transformar_dados(dados):
    df = pd.DataFrame(dados)

    # Renomear colunas para um padrão mais claro
    df.rename(columns={
        "id": "crime_id",
        "tipo": "tipo_crime",
        "data": "data_ocorrencia",
        "localizacao": "endereco",
        "descricao": "descricao_crime"
    }, inplace=True)

    # Converter colunas para o formato adequado
    df["data_ocorrencia"] = pd.to_datetime(df["data_ocorrencia"])

    return df

In [4]:
# Função para salvar os dados no PostgreSQL
def salvar_no_postgres(df):
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()

        # Criar tabela se não existir
        cur.execute("""
        CREATE TABLE IF NOT EXISTS crimes (
            crime_id SERIAL PRIMARY KEY,
            tipo_crime VARCHAR(255),
            data_ocorrencia TIMESTAMP,
            endereco TEXT,
            descricao_crime TEXT
        );
        """)

        # Inserir dados no banco
        for _, row in df.iterrows():
            cur.execute("""
            INSERT INTO crimes (tipo_crime, data_ocorrencia, endereco, descricao_crime)
            VALUES (%s, %s, %s, %s);
            """, (row["tipo_crime"], row["data_ocorrencia"], row["endereco"], row["descricao_crime"]))

        conn.commit()
        cur.close()
        conn.close()
        print("Dados inseridos no PostgreSQL com sucesso!")

    except Exception as e:
        print(f"Erro ao salvar no PostgreSQL: {e}")


In [5]:
# Fluxo do pipeline
dados_crimes = extrair_dados_api()
if dados_crimes:
    df_crimes = transformar_dados(dados_crimes)
    salvar_no_postgres(df_crimes)

Erro ao acessar API: 401
